In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [0]:
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table

In [0]:
%%bigquery --project cs224w-257119 issue_comment_df
SELECT
  JSON_EXTRACT_SCALAR(payload, '$.comment.id') as id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.body') as comment,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.id') as commenter_id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.login') as commenter_login,
  #JSON_EXTRACT_SCALAR(payload, '$.issue.id') as pr_id,
  JSON_EXTRACT_SCALAR(payload, '$.issue.number') as number,
  JSON_EXTRACT_SCALAR(payload, '$.issue.user.id') as user_id,
  JSON_EXTRACT_SCALAR(payload, '$.issue.user.login') as username,
  JSON_EXTRACT_SCALAR(payload, '$.issue.title') as title,
  JSON_EXTRACT_SCALAR(payload, '$.issue.body') as description,
  JSON_EXTRACT_SCALAR(payload, '$.issue.created_at') as pr_created_at,
  JSON_EXTRACT_SCALAR(payload, '$.comment.created_at') as comment_created_at
FROM `githubarchive.year.20*`
WHERE
  _TABLE_SUFFIX BETWEEN '15' AND '18'
  AND type = 'IssueCommentEvent'
  AND repo.name = "kubernetes/kubernetes"
  AND JSON_EXTRACT_SCALAR(payload, '$.action')="created"
  AND JSON_EXTRACT_SCALAR(payload, '$.issue.number') IN 
    (SELECT  DISTINCT
    JSON_EXTRACT_SCALAR(payload, '$.issue.number') as number
    FROM `githubarchive.year.20*`)


In [0]:
%%bigquery --project cs224w-257119 pr_comment_df

SELECT
  JSON_EXTRACT_SCALAR(payload, '$.comment.id') as id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.body') as comment,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.id') as commenter_id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.login') as commenter_login,
  #JSON_EXTRACT_SCALAR(payload, '$.pull_request.id') as pr_id,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.number') as number,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.id') as user_id,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.login') as username,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.title') as title,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.body') as description,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.created_at') as pr_created_at,
  JSON_EXTRACT_SCALAR(payload, '$.comment.created_at') as comment_created_at
FROM `githubarchive.year.20*`

WHERE
  _TABLE_SUFFIX BETWEEN '15' AND '18'
  AND type='PullRequestReviewCommentEvent'
  AND repo.name = "kubernetes/kubernetes"
  AND JSON_EXTRACT_SCALAR(payload, '$.action')="created"


In [0]:
comment_df = pr_comment_df.append(issue_comment_df)